**Ссылка**, на источник текста

In [ ]:
DATA_URL = 'http://az.lib.ru/g/gogolx_n_w/text_0050.shtml'

Устанавливаем библиотеки

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip install -q tensorflow==2.12
!pip install -q git+https://github.com/dvolchek/rnnmorph.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.15 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
bigframes 1.17.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatib

Создаём объект морфологического анализатора `RNNMorph`

In [ ]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [ ]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [ ]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Гоголь Николай Васильевич. Вий</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/g/gogolx_n_w/>Гоголь Николай Васильевич</a><br>\r\nВий</h2>\r\n\r\n<!------- П'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [ ]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Гоголь Николай Васильевич. Вий\n\n\n\nГоголь Николай Васильевич\r\nВий\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n[Помощь]\r\n\r\n\n\n'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


"A total of 934 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [ ]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent]
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
predictions[-11:-10] #Сейчас видно, что токены типа "точка", "запятая" и тд пока присутствуют в предложениях. От них нужно избавиться

15/15 [==============================] - 8s 432ms/step


sentences: 100%|██████████| 934/934 [00:00<00:00, 232546.59it/s]


[['пфейфер', '(', 'он', '.', ')']]

In [ ]:
predictions = [[token for token in sent if token.isalpha()] for sent in predictions]

In [ ]:
print(f'Количество предложений: {len(predictions)}')

Количество предложений: 934


In [ ]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
print(f'Количество токенов: {len(non_uniq_tokens)}')

Количество токенов: 11732


## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 50 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам.

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75.

In [ ]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['и', 'в', 'во', 'не', 'что']

In [ ]:
# получим 50 самых частотных токенов
fdist = FreqDist(word.lower() for word in non_uniq_tokens)
most_common_50 = fdist.most_common(50)
most_common_50

[('он', 553),
 ('и', 528),
 ('в', 322),
 ('на', 214),
 ('не', 191),
 ('быть', 191),
 ('что', 189),
 ('с', 136),
 ('философ', 125),
 ('как', 122),
 ('весь', 95),
 ('свой', 93),
 ('я', 85),
 ('по', 79),
 ('сказать', 69),
 ('а', 68),
 ('из', 66),
 ('к', 63),
 ('только', 61),
 ('за', 57),
 ('тот', 57),
 ('один', 56),
 ('но', 56),
 ('такой', 55),
 ('бы', 55),
 ('который', 54),
 ('или', 53),
 ('ты', 53),
 ('уже', 52),
 ('то', 50),
 ('у', 46),
 ('же', 45),
 ('это', 45),
 ('сам', 44),
 ('так', 42),
 ('другой', 40),
 ('когда', 40),
 ('да', 39),
 ('себя', 38),
 ('рука', 38),
 ('хома', 38),
 ('этот', 36),
 ('всё', 36),
 ('ни', 35),
 ('знать', 34),
 ('мочь', 34),
 ('о', 33),
 ('глаз', 32),
 ('говорить', 30),
 ('начать', 29)]

In [ ]:
# получим токены, не относящиеся к стоп-словам
non_stopword_tokens = [token for token, frequency in most_common_50 if token not in STOPWORDS]

# рассчитаем долю
non_stopword_tokens_fraction = len(non_stopword_tokens) / 50
print(f'Доля токенов, не относящихся к стоп словам среди 50 самых частотных токенов: {non_stopword_tokens_fraction:.2f}')

Доля токенов, не относящихся к стоп словам среди 50 самых частотных токенов: 0.28


## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

In [ ]:
tokens_above_50 = [token for token, frequency in fdist.items() if frequency > 50]
tokens_above_50_count = len(tokens_above_50)
print(f'Кол-во токенов, встречающихся в тексте строго больше 50 раз: {tokens_above_50_count}')

Кол-во токенов, встречающихся в тексте строго больше 50 раз: 29
